# Define podcast

In [ ]:
page_name = "Common Blackbird"
podcast_name = "Birds are awsome!"
podcast_slug = "common_blackbird"

# Generate podcast text

In [ ]:
import src.generate_text as gt

### Getting data

I use [wikipedia](www.wikie.com) to get the contents of a page. The text can come from anywhere, you could for instance get the data from a pdf or a book.

In [ ]:
input_text = gt.get_page_content_from_wikipedia(page_name)
texts = gt.split_text_into_documents(input_text)
texts

### Creating dialogue Script

Here we use OpenAI ChatGPT-3.5-turbo throught langchain to generate the script based on a given text input.

We use a sequential chain with the following chains:
-  Summarisation chain: create a summarisation chain using load_summarize_chain function from langchain with map_reduce chain type. This chain recieves chunks of text and create a summary for each step (map step) then concatenates all summaries in a single, concise summary (reduce step).
- Dialogue chain: create a function chain using create_structured_output_chain function from langchain. This will recieve a text summary and will generate a podcast dialogue between two people from it. The output will be a python dict with the dialogue.

In [ ]:
output_dialogue = gt.create_podcast_dialogue_from_text(input_text, podcast_name, verbose=False)
podcast_dialogues = output_dialogue["podcast_dialogues"]
display(podcast_dialogues)

# Audio generation

In [ ]:
import src.generate_audio as ga
import os.path

### Text-to-speech

Use elevenlabs to generate voices given a podcast dialogue.

In [ ]:
podcast_dialogues_sample = podcast_dialogues[:2] # sampling only initial dialogue to test
podcast_dialogues_sample

In [ ]:
ga.generate_voices(podcast_dialogues_sample,f"audios/episodes/ep_{podcast_slug}.mp3")

### Creating into song

Use MusicGen from Audiocraft to generate a 15s song for the intro.

In [ ]:
if not os.path.isfile("audios/intro_main_song.wav"):
    description = "earthy tones, environmentally conscious, ukulele-infused, harmonic, breezy, easygoing, organic instrumentation, gentle grooves"
    audio = ga.create_audio_sample(description, save_filename="audios/intro_main_song")
else:
    print("Intro song already created")

### Creating sound effecs for intro

Use AudioGen from audiocraft to generate a sound effect that will be added as a background sound to the intro.

In [ ]:
if not os.path.isfile("audios/intro_bird_effect.wav"):
    description = "sounds of birds"
    effects = ga.create_effects_sample(description, save_filename="audios/intro_bird_effect")
else:
    print("Intro effect already created")

### All together

Mix intro sound with effects and concatenate with the podcast audio.

In [ ]:
full_episode_audio = ga.add_into_ontro_podcast_audio(f"audios/episodes/ep_{podcast_slug}.mp3")
full_episode_audio.export(f"audios/full_episodes/ep_{podcast_slug}.mp3", format="mp3")

# Image generation

In [ ]:
import src.generate_images as gi

### Generate cover image

Generate a image using stable-diffusion-xl-base-1.0 as base model and stable-diffusion-xl-refiner-1.0 as a refiner model.

In [ ]:
prompt_pop_art = f"Pop art illustration of a {page_name}, comic book-inspired, vivid and contrasting colors, layered composition, retro flair, lively and expressive"
negative_prompt = "multiple birds, out of frame, lowres, text, error, cropped, worst quality, low quality, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature"

image = gi.generate_cover(prompt_pop_art, negative_prompt)
gi.print_podcast_cover(image, page_name)